In [ ]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd

In [ ]:
adata_rna = sc.read('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/multiome_gex_processed_cell_typed_raw.gz.h5ad')
adata_rna

In [ ]:
muon_adata = mu.read_10x_h5("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/filtered_feature_bc_matrix.h5")

In [ ]:
muon_adata = muon_adata[adata_rna.obs_names,:]
mu_rna = muon_adata['rna'].copy()
mu_rna.var_names_make_unique()
mu_rna = mu_rna[:,adata_rna.var_names]
mu_rna.layers['soupx'] = adata_rna.X.copy()
mu_rna.obs = adata_rna.obs[['mouse', 'sex', 'treatment', 'timepoint','lineage', 'celltype']]


mu_atac = muon_adata['atac'].copy()
mu_atac.obs = adata_rna.obs[['mouse', 'sex', 'treatment', 'timepoint','lineage', 'celltype']]
mu_atac.uns['atac'] = dict(mu_atac.uns['atac']) # anndata does not write ordered dict
mu_atac.uns['files'] = dict(mu_atac.uns['files']) # anndata does not write ordered dict

In [ ]:
mu_rna.obs.to_csv('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_metadata.csv')

In [ ]:
mu_rna.write('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_rna.gz.h5ad',
                       compression='gzip')
mu_atac.write('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_atac.gz.h5ad',
                       compression='gzip')

In [ ]:
mu_atac.uns['files']

In [ ]:
sc.pl.dotplot(adata_rna, ['Cxcl10'], groupby=['celltype','treatment'])

In [ ]:
mu_rna.layers['soupx']

In [ ]:
adata_rna.obs[['mouse', 'sex', 'treatment', 'timepoint','lineage', 'celltype']]

In [ ]:
muon_adata.obs

In [ ]:
muon_adata.var_names

In [ ]:
adata_rna

In [ ]:
adata_rna

In [ ]:
adata_atac = muon_adata['atac']
adata_atac.var_names

In [ ]:
# tf-bc matrix
import scipy.io
import csv
import os 
matrix_dir = "/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/outs/analysis/tf_analysis/filtered_tf_bc_matrix"
adata_tf = sc.read_mtx(os.path.join(matrix_dir, "matrix.mtx"))
 
motifs_path = os.path.join(matrix_dir, "motifs.tsv")
var = pd.read_csv(motifs_path, sep='\t', index_col =0,header=None)
barcodes_path = os.path.join(matrix_dir, "barcodes.tsv")
obs = pd.read_csv(barcodes_path, sep='\t', index_col =0,header=None)
adata_tf = adata_tf.T
adata_tf.layers['raw']= adata_tf.X.copy()
adata_tf.obs_names=obs.index.values
adata_tf.var_names=var.index.values


adata_tf = adata_tf[adata_rna.obs_names]
adata_tf.obs = adata_rna.obs[['celltype','mouse','treatment','lineage']]


adata_tf.X = adata_tf.layers['raw'].copy()
# sc.pp.normalize_total(adata_tf)
# sc.pp.log1p(adata_tf)
sc.tl.rank_genes_groups(adata_tf, groupby ='celltype', method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(adata_tf, groupby='celltype', n_genes=1)


In [ ]:
tf_bed = pd.read_csv("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/outs/analysis/tf_analysis/peak_motif_mapping.bed",
                     sep='\t',
                     index_col=None,
                     header=None
                    )


In [ ]:
tf_bed

In [ ]:
adata_atac.uns['atac']['peak_annotation'].loc['Slc6a2']

In [ ]:
var[1].loc[var[1].str.startswith('Sox')]